# Build a Reflective Dynamic Planning Agent for Multi-Step Complex Query Analysis with LangGraph

In this project we will be building a Planning Agent which can generate dynamic plans and iterate and update using reflection to help with multi-step complex query analysis.

![](https://i.imgur.com/odnq3de.png)

### Reflective Dynamic Planning Agent for Multi-Step Complex Query Analysis

This project focuses on building a **Reflective Dynamic Planning Agent for Multi-Step Complex Query Analysis**, designed to break down user queries or tasks into manageable steps, execute them iteratively, and update the plan dynamically through reflection and replanning. The components of the system are as follows:

1. **Plan Creation**:
   - The agent begins by analyzing the user's input (topic or question) to create a **step-by-step plan** of sub-questions or steps needed to solve the overall task.
   - Each step is clearly outlined in the **step-by-step plan** for sequential execution.

2. **Step Execution**:
   - The agent executes one step at a time using **OpenAI GPT-4o**.
   - During execution:
     - External tools, such as the **web search tool**, are used to gather required information or data for the step.
     - The result of the executed step is captured and stored in the agent's state.

3. **Reflection and Replanning**:
   - After each step, the system evaluates the results and sends the **overall plan**, **steps executed**, **results**, and **remaining steps** to the **Replanner**.
   - The **Replanner**:
     - Assesses whether the remaining steps need modification based on the current results and any new insights.
     - Updates the remaining steps if necessary, ensuring alignment with the overall objective.
   - This iterative loop ensures adaptability to errors, incomplete data, or unforeseen circumstances.

4. **Dynamic Feedback Loop**:
   - The system monitors the status of the response:
     - If all steps are executed successfully or an error occurs, the process halts.
     - If there are remaining steps, the agent continues executing them one at a time, integrating results dynamically.

5. **Agent State Maintenance**:
   - Throughout the process, the agent maintains a record of:
     - The **original plan**.
     - **Executed steps and their results**.
     - **Remaining steps** for further execution.
   - This stateful approach ensures traceability and transparency in the agent’s decision-making process.

6. **Final Response Compilation**:
   - Once all steps are executed or the process encounters an error, the agent compiles everything to generate the final reponse.





## Install OpenAI, LangGraph and LangChain dependencies

In [0]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langgraph==0.2.64
!pip install rich

In [0]:
!pip install markitdown

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [0]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Utility Functions

- __`process_and_truncate_texts(...)`:__ Helps in truncating the content length of content extracted from the web (documents), this is very useful especially to keep documents within the limit of LLM context windows

In [0]:
import tiktoken

def process_and_truncate_texts(text_docs, max_tokens=127000):
    """
    Calculate token counts for a list of text documents and truncate them to fit within max_tokens.

    Args:
        text_docs (list of str): List of text documents to process.
        max_tokens (int): Maximum token limit for truncation.

    Returns:
        list of str: Truncated list of text documents.
    """
    # Load the tokenizer (adjust the encoding name as per your use case, e.g., gpt-3.5-turbo, gpt-4, etc.)
    encoding = tiktoken.encoding_for_model("gpt-4o")

    # Calculate token counts for each document
    tokenized_docs = []
    for text in text_docs:
        tokens = encoding.encode(text)
        tokenized_docs.append(tokens)

    # Flatten all tokens into a single list
    all_tokens = [token for tokens in tokenized_docs for token in tokens]

    # Truncate to the maximum token limit
    truncated_tokens = all_tokens[:max_tokens]

    # Decode the truncated tokens back to text
    truncated_texts = []
    remaining_tokens = truncated_tokens[:]
    for tokens in tokenized_docs:
        if len(remaining_tokens) >= len(tokens):
            # Add the full document if its tokens fit within the remaining budget
            truncated_texts.append(encoding.decode(tokens))
            remaining_tokens = remaining_tokens[len(tokens):]
        else:
            # Partially add tokens of the last fitting document
            truncated_texts.append(encoding.decode(remaining_tokens))
            break

    return truncated_texts

text_docs = [
        "This is the first document. It has some content.",
        "The second document might have more content than the first.",
        "This is the third document. It also has its own content.",
        # Add more documents as needed
    ]

truncated_texts = process_and_truncate_texts(text_docs, max_tokens=20)
truncated_texts

- __`search_web_extract_info(...)`:__ Leverages Tavily to get top search results for a query and then uses Tavily extracted raw content to get the full website data and then calls `process_and_truncate_texts(...)` on the extracted documents to truncate them to a specific token limit

In [0]:
from langchain_core.tools import tool
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper

tavily_search = TavilySearchAPIWrapper()


def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool for query:', query)
    results = tavily_search.raw_results(query=query,
                                        max_results=3,
                                        search_depth='advanced',
                                        include_answer=False,
                                        include_raw_content=True)
    docs = [doc['raw_content'] for doc in results['results']]
    docs = process_and_truncate_texts(docs, max_tokens=127000)
    return docs

In [0]:
docs = search_web_extract_info('what is langgraph')

In [0]:
len(docs)

In [0]:
docs[0]

## Build the Web Research Sub-Agent graph - Tool-Use Agent

This agent or to be more accurate sub-agent will be used in our planner to get information for each query or step in the overall plan

In [0]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage, RemoveMessage
from langchain_core.messages import trim_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tools = [search_web_extract_info]
llm = ChatOpenAI(model="gpt-5")
llm_with_tools = llm.bind_tools(tools)



SYS_MSG = SystemMessage(content="Act as a helpful assistant and answer user questions")
def chatbot(state: State):
    messages = trim_messages(
            state["messages"],
            max_tokens=127000,
            strategy="last",
            token_counter=ChatOpenAI(model="gpt-4o"),
            allow_partial=True,
    )
    return {"messages": [llm_with_tools.invoke([SYS_MSG] + messages)]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
    ["tools", END]
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
react_agent = graph_builder.compile()

In [0]:
from IPython.display import Image, display

display(Image(react_agent.get_graph().draw_mermaid_png()))

## Test the Web Research Sub-Agent

In [0]:
def call_agent(agent, prompt, user_config={"configurable": {"thread_id": "any"}}):
    events = agent.stream(
        {"messages": [{"role": "user", "content": prompt}]},
        user_config,
        stream_mode="values",
    )

    for event in events:
        event["messages"][-1].pretty_print()

In [0]:
user_prompt = "Can you tell me about langgraph?"

call_agent(agent=react_agent,
           prompt=user_prompt)

In [0]:
user_prompt = "Who are the key founders of Nvidia, what did they previously do and how did nvidia become such a huge company with a huge market cap?"

call_agent(agent=react_agent,
           prompt=user_prompt)

In [0]:
user_prompt = """who is the founder of microsoft, their hometown and history of founder and the company?
               """

call_agent(agent=react_agent,
           prompt=user_prompt)

## Create State Schema for our Planning Agent

In [0]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict

# state of the executed plan including overall plan, current status in terms of steps executed and output
# response would either be the final result or errors encountered
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

## Create AI Workflow for Plan Generation

In [0]:
from pydantic import BaseModel, Field
from typing import Literal, Union

class Plan(BaseModel):
    """Plan to follow in future to get to a well-researched output for user query"""
    type: Literal["Plan"]
    steps: List[str] = Field(
        description="Different steps to follow in the plan, should be in sorted order"
    )

In [0]:
from langchain_core.prompts import ChatPromptTemplate

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Analyze the given query or task in detail and follow these rules to make a plan:
                - Come up with a simple step by step plan (max 5 steps).
                - This plan should involve individual tasks, that if executed correctly will yield the correct answer.
                - Do not add any extra superfluous steps.
                - The result of the final step should lead to the final answer.
                - Make sure that each step has all the information needed - do not skip steps.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

In [0]:
llm_planner = ChatOpenAI(model="gpt-4o-mini", temperature=0)
planner = (
            planner_prompt
                |
            llm_planner.with_structured_output(Plan) #function calling because we are using structured output
)

In [0]:
res = planner.invoke(
    {
        "messages": [
            ("user", "who is the founder of microsoft")
        ]
    }
)

print(res)

In [0]:
res

## Create AI Workflow for Replanner (with reflection)

In [0]:
from typing import Union

class Response(BaseModel):
    """Response generated for the user."""
    type: Literal["Response"]
    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )

In [0]:
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan.
       This plan should involve individual tasks, that if executed correctly will yield the correct answer.
       Do not add any extra superfluous steps.
       The result of the final step should be the final answer.
       Make sure that each step has all the information needed - do not skip steps.

       Your main objective was this:
       {input}

       Your original plan was this:
       {plan}

       You have currently done the following steps (with their corresponding results):
       {past_steps}

       Update your plan accordingly as necessary.

       If no more steps are needed then analyze all the information collected so far and generate a detailed formatted answer in Markdown.
       When generating the final response in markdown, if there are special characters in the text, such as the dollar symbol,
       ensure they are escaped properly for correct rendering e.g $25.5 should become \$25.5

       Otherwise, fill out the plan.
       Only add steps to the plan that still NEED to be done.
       Do NOT return previously done steps as part of the plan."""
)

llm_replanner = ChatOpenAI(model="gpt-4o-mini", temperature=0)

replanner = (
                replanner_prompt
                    |
                llm_replanner.with_structured_output(Act)
)

## Create Node Function for Plan Generation

Uses the Planner AI Workflow from above to generate a plan for the given user query

In [0]:
async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}

## Create Node Function for Plan Step Execution

Executes the current step in the plan using the researcher sub-agent

In [0]:
react_agent = graph_builder.compile() # compile and prepare the researcher sub-agent

async def execute_step(state: PlanExecute):
    plan = state["plan"]
    past_steps = state['past_steps']
    if not plan:  # Check if plan is empty
        return {
            "past_steps": [],
            "response": "No steps to execute in the plan."
        }

    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    print('=== plan str ===')
    print(plan_str)
    print('=== end of plan str ===')
    task = plan[0]
    task_formatted = f"""Given the following plan:
                        {plan_str}

                         You are tasked with executing step {1}, {task} only.

                         Here are the past steps already executed, use this information also as needed.
                         {past_steps}

                         Call tools only when you need more information.
                         If doing web search optimize the query for search before tool calling
                         otherwise refer to all the information you have to solve the step.
                         Do NOT call tools for analyzing existing information or compiling reports
                         or formatting documents.
                         """

    agent_response = await react_agent.ainvoke(
        {"messages": [("user", task_formatted)]}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }

## Create Node Function for Replanner

Updates the current plan's next steps if needed by reflecting on the past steps executed using the Replanner AI workflow from above

In [0]:
async def replan_step(state: PlanExecute):
    try:
        output = await replanner.ainvoke(state)
        print('=== replanner output ===')
        print(output)
        print('=== end of replanner output ===')

        # Handle dict response
        if isinstance(output, dict) and 'Response' in output:
            return {"response": output['Response']}

        # Handle proper Act instance
        if isinstance(output.action, Response):
            return {"response": output.action.response}
        else:
            return {"plan": output.action.steps}
    except Exception as e:
        print(f"Error in replan_step: {e}")
        return {"response": str(output.get('Response', "An error occurred processing the response."))}

## Create Conditional Edge Function for Stopping the Agent

Stops the agent once all steps have been executed or there is an error

In [0]:
from langgraph.graph import END

def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "react_agent"

## Build Planner Agent Graph

In [0]:
from langgraph.graph import StateGraph, START

plan_graph = StateGraph(PlanExecute)

# Add the plan node
plan_graph.add_node("planner", plan_step)
# Add the execution step
plan_graph.add_node("react_agent", execute_step)
# Add a replan node
plan_graph.add_node("replanner", replan_step)

# From plan we go to agent
plan_graph.add_edge("planner", "react_agent")
# From agent, we replanner
plan_graph.add_edge("react_agent", "replanner")
plan_graph.add_conditional_edges(
    "replanner",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["react_agent", END]
)
plan_graph.set_entry_point("planner")
# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
planner_agent = plan_graph.compile()

In [0]:
display(Image(planner_agent.get_graph(xray=True).draw_mermaid_png()))

## Run and Test Planner Agent

In [0]:
from IPython.display import display
from rich.markdown import Markdown as RichMarkdown


async def call_planner_agent(agent, prompt, config={"recursion_limit": 50}, verbose=False):
    events = agent.astream(
        prompt,
        config,
        stream_mode="values",
    )

    async for event in events:
        for k, v in event.items():
            if verbose:
                if k != "__end__":
                    display(RichMarkdown(repr(k) + ' -> ' + repr(v)))
            if k == 'response':
                print('='*50)
                print('Response:')
                display(RichMarkdown(v))


In [0]:
prompt = {"input": "who is the founder of microsoft, their hometown and history of founder and the company?"}
await call_planner_agent(agent=planner_agent,
                         prompt=prompt,
                         verbose=True)

In [0]:
prompt = {"input": "Who are the key founders of Nvidia, what did they previously do and how did nvidia become such a huge company with a huge market cap?"}
await call_planner_agent(agent=planner_agent,
                         prompt=prompt)